# Advanced LLM features

In [1]:
import base64
from pathlib import Path
from pprint import pp
import json
import logging

import openai
from openai import OpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from IPython.display import display, Image, Audio
import requests

load_dotenv()

client = OpenAI()

## 1. Sctructured output

## Преимущества 

1. **Надежная типизация**: Нет необходимости валидации или повторной попытки обработки некорректно отформатированных ответов.
2. **Обработка ошибок**: Отказы модели на основе безопасности теперь можно обнаружить программным способом.
3. **Проще формулировать запросы**: Нет необходимости в развернутых формулировках для достижения ожидаемого формата ответа.

In [2]:
output_schema = {
  "name": "course_structure",
  "schema": {
    "type": "object",
    "properties": {
      "course_title": {
        "type": "string",
        "description": "The title of the course."
      },
      "course_description": {
        "type": "string",
        "description": "A brief description of the course."
      },
      "modules": {
        "type": "array",
        "description": "List of modules in the course.",
        "items": {
          "type": "object",
          "properties": {
            "module_number": {
              "type": "number",
              "description": "The sequential number of the module."
            },
            "module_title": {
              "type": "string",
              "description": "The title of the module."
            },
            "module_description": {
              "type": "string",
              "description": "A brief description of the module."
            },
            "lessons": {
              "type": "array",
              "description": "List of lessons within the module.",
              "items": {
                "type": "object",
                "properties": {
                  "lesson_number": {
                    "type": "number",
                    "description": "The sequential number of the lesson."
                  },
                  "lesson_title": {
                    "type": "string",
                    "description": "The title of the lesson."
                  },
                  "lesson_description": {
                    "type": "string",
                    "description": "A brief description of the lesson."
                  }
                },
                "required": [
                  "lesson_number",
                  "lesson_title",
                  "lesson_description"
                ],
                "additionalProperties": False
              }
            }
          },
          "required": [
            "module_number",
            "module_title",
            "module_description",
            "lessons"
          ],
          "additionalProperties": False
        }
      }
    },
    "required": [
      "course_title",
      "course_description",
      "modules"
    ],
    "additionalProperties": False
  },
  "strict": True
}

In [5]:
messages = [{'role':'system', 'content':'Ты - ИИ помощник методиста и преподавателя онлайн-курсов. Ты помогаешь разрабоатывать программы и онлайн-курсы'},
            {'role':'user', 'content': 'Разработай программу курса из 1 модуля и 5 уроков по введению в ИИ для представителей государственных органов'}]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    response_format={"type": "json_schema", "json_schema":output_schema}
)

text_response = response.choices[0].message.content
json_response = json.loads(text_response)
pp(json_response)

{'course_title': 'Введение в Искусственный Интеллект для Представителей '
                 'Государственных Органов',
 'course_description': 'Этот курс предназначен для представителей '
                       'государственных органов, чтобы дать им общее '
                       'представление об искусственном интеллекте (ИИ), его '
                       'применении и возможностях для государственной сферы. '
                       'Курс также ознакомит с концепциями, этическими '
                       'аспектами и политическими последствиями внедрения ИИ в '
                       'государственный сектор.',
 'modules': [{'module_number': 1,
              'module_title': 'Основы Искусственного Интеллекта',
              'module_description': 'В этом модуле слушатели узнают об '
                                    'основных концепциях ИИ, его истории и '
                                    'текущих трендах. Будет рассмотрено, как '
                                    'ИИ может быть ис

In [ ]:
# chain-of-thougth

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "Сколько букв r в слове strrawberry?"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed
print(math_reasoning)

In [ ]:
# User messages classification

from enum import Enum
from typing import Optional

class Category(str, Enum):
    help_desk = "helpdesk"
    sales_promo = "sales promo"
    start_buy = "start to buy"

class UserIntent(BaseModel):
    is_start_buy: bool
    category: Optional[Category]
    explanation_if_start_buy: Optional[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Determine user intent and explain if they do start buy."},
        {"role": "user", "content": "Мне нужны кроссовки 42 размера, в каком разделе мне их искать?"}
    ],
    response_format=UserIntent,
)

user_intent = completion.choices[0].message.parsed
print(user_intent)

## 2. Function calling

<img src="./images/function_callin_main.png" alt="Drawing" style="width: 800px;"/>

In [7]:
def find_tutor(subject: str, start_date: str, num_sessions: int) -> list:
    # some code here
    # ...
    return [('Иванов Семен', 'ivanov@semen'), ('Петров Николай', 'petrov@nikolay')]

function_description = {
  "name": "find_tutor",
  "description": "Функция выполняет поиск свободного репетитора.",
  "strict": True,
  "parameters": {
    "type": "object",
    "required": [
      "subject",
      "start_date",
      "num_sessions"
    ],
    "properties": {
      "subject": {
        "type": "string",
        "description": "Предмет, по которому требуется найти репетитора"
      },
      "start_date": {
        "type": "string",
        "description": "Дата начала занятий в формате YYYY-MM-DD"
      },
      "num_sessions": {
        "type": "number",
        "description": "Требуемое количество занятий"
      }
    },
    "additionalProperties": False
  }
}

tools = [
    {
        "type": "function",
        "function": function_description
    }
]

messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Привет, я хотел бы найти репетитора по английскому языку. Подскажите, кто из репетиторов сейчас доступен?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.choices[0].message.content)

Чтобы помочь вам найти доступного репетитора по английскому языку, мне необходимо знать дату начала занятий и количество занятий, которое вам требуется. Пожалуйста, предоставьте эту информацию.


In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Привет, я хотел бы найти репетитора по английскому языку. Подскажите, кто из репетиторов сейчас доступен?"
    },
    {
        "role": "assistant",
        "content": "Чтобы помочь вам найти доступного репетитора по английскому языку, мне необходимо знать дату начала занятий и количество занятий, которое вам требуется. Пожалуйста, предоставьте эту информацию."
    },
    {
        "role": "user",
        "content": "с 22 января, нужно я думаю не меньше 10 занятий"
    }
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.choices[0])

tool_call = response.choices[0].message.tool_calls[0]
print(tool_call)
arguments = json.loads(tool_call.function.arguments)
tutors = find_tutor(**arguments)
print(tutors)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_O2hA8A0apRq1LCqr6elqtV1I', function=Function(arguments='{"subject":"английский язык","start_date":"2024-01-22","num_sessions":10}', name='find_tutor'), type='function')]))
ChatCompletionMessageToolCall(id='call_O2hA8A0apRq1LCqr6elqtV1I', function=Function(arguments='{"subject":"английский язык","start_date":"2024-01-22","num_sessions":10}', name='find_tutor'), type='function')
[('Иванов Семен', 'ivanov@semen'), ('Петров Николай', 'petrov@nikolay')]


In [ ]:
function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "avaliable_tutors": tutors
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
}

messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Привет, я хотел бы найти репетитора по английскому языку. Подскажите, кто из репетиторов сейчас доступен?"
    },
    {
        "role": "assistant",
        "content": "Чтобы помочь вам найти доступного репетитора по английскому языку, мне необходимо знать дату начала занятий и количество занятий, которое вам требуется. Пожалуйста, предоставьте эту информацию."
    },
    {
        "role": "user",
        "content": "с 22 января, нужно я думаю не меньше 10 занятий"
    },
    response.choices[0].message,
    function_call_result_message
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.choices[0].message.content)

Сейчас доступны следующие репетиторы по английскому языку:

1. Иванов Семен - электронная почта: ivanov@semen
2. Петров Николай - электронная почта: petrov@nikolay

Вы можете связаться с ними, чтобы обсудить детали занятий. Если у вас есть другие вопросы, не стесняйтесь спрашивать!


In [30]:
class GetAvaliableTutors(BaseModel):
    '''Функция выполняет поиск свободного репетитора'''
    subject: str = Field(description='Предмет, по которому требуется найти репетитора')
    start_date: str = Field(description='Дата начала занятий в формате YYYY-MM-DD')
    num_sessions: int = Field(description='Требуемое количество занятий')
     
tools = [openai.pydantic_function_tool(GetAvaliableTutors)]

print(tools)

[{'type': 'function', 'function': {'name': 'GetAvaliableTutors', 'strict': True, 'parameters': {'description': 'Функция выполняет поиск свободного репетитора', 'properties': {'subject': {'description': 'Предмет, по которому требуется найти репетитора', 'title': 'Subject', 'type': 'string'}, 'start_date': {'description': 'Дата начала занятий в формате YYYY-MM-DD', 'title': 'Start Date', 'type': 'string'}, 'num_sessions': {'description': 'Требуемое количество занятий', 'title': 'Num Sessions', 'type': 'integer'}}, 'required': ['subject', 'start_date', 'num_sessions'], 'title': 'GetAvaliableTutors', 'type': 'object', 'additionalProperties': False}, 'description': 'Функция выполняет поиск свободного репетитора'}}]


In [31]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Привет, я хотел бы найти репетитора по английскому языку. Подскажите, кто из репетиторов сейчас доступен?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.choices[0].message.content)

Когда вы планируете начать занятия и сколько сессий вам потребуется?


In [40]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Привет, я хотел бы найти репетитора по английскому языку. Подскажите, кто из репетиторов сейчас доступен?"
    },
    {
        "role": "assistant",
        "content": "Когда вы планируете начать занятия и сколько сессий вам потребуется?"
    },
    {
        "role": "user",
        "content": "с 22 января, нужно я думаю не меньше 10 занятий"
    },
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"subject":"английский","start_date":"2024-01-22","num_sessions":10}', name='GetAvaliableTutors')


In [ ]:
# openai.log = "debug"
# logging.basicConfig(level=logging.DEBUG,
#                     format='%(asctime)s - %(levelname)s - %(filename)s:%(funcName)s:%(lineno)d - %(message)s',
#                     datefmt='%Y-%m-%d %H:%M:%S')


arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
tutors = find_tutor(**arguments)

function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "avaliable_tutors": tutors
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
}

messages.append(response.choices[0].message)
messages.append(function_call_result_message)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.choices[0].message.content)

2024-11-22 13:12:45 - DEBUG - _base_client.py:_build_request:453 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a helpful customer support assistant. Use the supplied tools to assist the user.'}, {'role': 'user', 'content': 'Привет, я хотел бы найти репетитора по английскому языку. Подскажите, кто из репетиторов сейчас доступен?'}, {'role': 'assistant', 'content': 'Когда вы планируете начать занятия и сколько сессий вам потребуется?'}, {'role': 'user', 'content': 'с 22 января, нужно я думаю не меньше 10 занятий'}, {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_8umQwOhwVDP6NoCZYZm3fU9A', 'function': {'arguments': '{"subject":"английский","start_date":"2024-01-22","num_sessions":10}', 'name': 'GetAvaliableTutors'}, 'type': 'function'}]}, {'role': 'tool', 'content': '{"avaliable_tutors": [["\\u0418\\u0432\\u0430\\u043d\\u043e\\u0432 \\u0421\\u0435\\u043c

2024-11-22 13:12:46 - DEBUG - _trace.py:trace:47 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 22 Nov 2024 10:12:46 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-vzydrwbzqejmtvohwohb3rlu'), (b'openai-processing-ms', b'1171'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'5000'), (b'x-ratelimit-limit-tokens', b'800000'), (b'x-ratelimit-remaining-requests', b'4999'), (b'x-ratelimit-remaining-tokens', b'799802'), (b'x-ratelimit-reset-requests', b'12ms'), (b'x-ratelimit-reset-tokens', b'14ms'), (b'x-request-id', b'req_9d968c6278343c40bf62fd496efd1455'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8e6824973afed602-IST'

Вот список доступных репетиторов по английскому языку с 22 января:

1. **Иванов Семен**: [ivanov@semen]
2. **Петров Николай**: [petrov@nikolay]

Вы можете связаться с ними по указанным адресам электронной почты для дальнейших обсуждений. Если у вас есть еще вопросы, дайте знать!
